In [1]:
import os
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split

## 집값 예측하기

In [3]:
# 데이터 불러오기
data = pd.read_csv("../3. 실습데이터/kc_house_data.csv") 
data.head() # 데이터 확인

,id,date,price,bedrooms,bathrooms,floors,waterfront,condition,grade,yr_built,yr_renovated,zipcode,lat,long
0,7129300520,20141013T000000,221900.0,3,1.00,1.0,0,3,7,1955,0,98178,47.5112,-122.257
1,6414100192,20141209T000000,538000.0,3,2.25,2.0,0,3,7,1951,1991,98125,47.7210,-122.319
2,5631500400,20150225T000000,180000.0,2,1.00,1.0,0,3,6,1933,0,98028,47.7379,-122.233
3,2487200875,20141209T000000,604000.0,4,3.00,1.0,0,5,7,1965,0,98136,47.5208,-122.393
4,1954400510,20150218T000000,510000.0,3,2.00,1.0,0,3,8,1987,0,98074,47.6168,-122.045


In [4]:
'''
id: 집 고유아이디
date: 집이 팔린 날짜 
price: 집 가격 (타겟변수)
bedrooms: 주택 당 침실 개수
bathrooms: 주택 당 화장실 개수
floors: 전체 층 개수
waterfront: 해변이 보이는지 (0, 1)
condition: 집 청소상태 (1~5)
grade: King County grading system 으로 인한 평점 (1~13)
yr_built: 집이 지어진 년도
yr_renovated: 집이 리모델링 된 년도
zipcode: 우편번호
lat: 위도
long: 경도
'''

'\nid: 집 고유아이디\ndate: 집이 팔린 날짜 \nprice: 집 가격 (타겟변수)\nbedrooms: 주택 당 침실 개수\nbathrooms: 주택 당 화장실 개수\nfloors: 전체 층 개수\nwaterfront: 해변이 보이는지 (0, 1)\ncondition: 집 청소상태 (1~5)\ngrade: King County grading system 으로 인한 평점 (1~13)\nyr_built: 집이 지어진 년도\nyr_renovated: 집이 리모델링 된 년도\nzipcode: 우편번호\nlat: 위도\nlong: 경도\n'

In [5]:
nCar = data.shape[0] # 데이터 개수(행갯수)
nVar = data.shape[1] # 변수 개수(열갯수)
print('nCar: %d' % nCar, 'nVar: %d' % nVar )

nCar: 21613 nVar: 14


## 의미가 없다고 판단되는 변수 제거

In [6]:
data = data.drop(['id', 'date', 'zipcode', 'lat', 'long'], axis = 1) # id, date, zipcode, lat, long  제거

## 범주형 변수를 이진형 변수로 변환
- 범주형 변수는 waterfront 컬럼 뿐이며, 이진 분류이기 때문에 0, 1로 표현한다.
- 데이터에서 0, 1로 표현되어 있으므로 과정 생략

## 설명변수와 타겟변수를 분리, 학습데이터와 평가데이터 분리

In [7]:
feature_columns = list(data.columns.difference(['price'])) # Price를 제외한 모든 행
X = data[feature_columns]
y = data['price']
train_x, test_x, train_y, test_y = train_test_split(X, y, test_size = 0.3, random_state = 42) # 학습데이터와 평가데이터의 비율을 7:3
print(train_x.shape, test_x.shape, train_y.shape, test_y.shape) # 데이터 개수 확인

(15129, 8) (6484, 8) (15129,) (6484,)


## 학습 데이터를 선형 회귀 모형에 적합 후 평가 데이터로 검증

In [8]:
import statsmodels.api as sm
from sklearn.metrics import mean_squared_error, r2_score
from math import sqrt

sm_train_x = sm.add_constant(train_x, has_constant = 'add') # Bias 추가
sm_model = sm.OLS(train_y, sm_train_x) # 모델 구축
fitted_sm_model = sm_model.fit() # 학습 진행
fitted_sm_model.summary() # 학습 모델 구조 확인

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                  price   R-squared:                       0.595
Model:                            OLS   Adj. R-squared:                  0.595
Method:                 Least Squares   F-statistic:                     2776.
Date:                Fri, 18 Sep 2020   Prob (F-statistic):               0.00
Time:                        00:41:28   Log-Likelihood:            -2.0826e+05
No. Observations:               15129   AIC:                         4.165e+05
Df Residuals:                   15120   BIC:                         4.166e+05
Df Model:                           8                                         
Covariance Type:            nonrobust                                         
================================================================================
                   coef    std err          t      P>|t|      [0.025      0.975]
--------------------------------------------------------------------------------
const         7.186e+06   1.73e+05     41.548      0.000    6.85e+06    7.52e+06
bathrooms     1.303e+05   3960.833     32.889      0.000    1.23e+05    1.38e+05
bedrooms     -2224.7910   2382.356     -0.934      0.350   -6894.497    2444.915
condition     1.641e+04   3169.013      5.178      0.000    1.02e+04    2.26e+04
floors        1946.3052   4336.838      0.449      0.654   -6554.422    1.04e+04
grade         1.956e+05   2199.540     88.924      0.000    1.91e+05       2e+05
waterfront    7.555e+05   2.26e+04     33.479      0.000    7.11e+05       8e+05
yr_built     -4300.7865     88.073    -48.832      0.000   -4473.420   -4128.153
yr_renovated    12.7325      5.043      2.525      0.012       2.847      22.618
==============================================================================
Omnibus:                    13447.374   Durbin-Watson:                   1.994
Prob(Omnibus):                  0.000   Jarque-Bera (JB):          1684794.827
Skew:                           3.763   Prob(JB):                         0.00
Kurtosis:                      54.147   Cond. No.                     1.82e+05
==============================================================================

Warnings:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
[2] The condition number is large, 1.82e+05. This might indicate that there are
strong multicollinearity or other numerical problems.
"""

#### R-squared: y의 총변동성/x가 설명하는 총변동성
##### 사용한 데이터로 집값을 설명하는 비율이 0.595 ==약 60퍼센트
#### bethroom coef 1.303 =양의 상관관계 화장실이 많을수록 집값이 오른다
#### bedrooms coef -2224 =음의상관관계 but p-value가 높으므로 ->귀무가설채택->회기계수는 0이다 즉, 변수의 설명력이 없다
#### 아마도 bethroom이랑 다중공선성 때문일것임

In [9]:
sm_test_x=sm.add_constant(test_x,has_constant="add")
sm_model_predict=fitted_sm_model.predict(sm_test_x)


In [10]:
sqrt(mean_squared_error(sm_model_predict,test_y)) #mse에 루트 rmse 적을수록 좋은것

239804.2967085816

# 선형회기로 Bagging하기
## Bagging한 결과가 일반적인 결과보다 좋은지 확인

학습데이터15129개

In [11]:
train_x

,bathrooms,bedrooms,condition,floors,grade,waterfront,yr_built,yr_renovated
167,2.50,4,3,2.0,9,0,1999,0
12412,1.75,4,5,1.0,7,0,1924,0
7691,2.25,4,3,2.0,8,0,1978,0
12460,1.75,2,3,2.0,9,0,2005,0
9099,1.75,4,4,1.0,8,0,1979,0
...,...,...,...,...,...,...,...,...
11964,1.50,3,3,1.0,7,0,1947,0
21575,2.50,3,3,2.0,8,0,2014,0
5390,2.50,3,3,2.0,7,0,2004,0
860,0.75,1,3,1.0,5,0,1963,0


In [23]:
print(train_x.iloc[0])

bathrooms          2.5
bedrooms           4.0
condition          3.0
floors             2.0
grade              9.0
waterfront         0.0
yr_built        1999.0
yr_renovated       0.0
Name: 167, dtype: float64


In [26]:
import random 
bagging_predict_result=[]
for _ in range(10):
    data_index=[data_index for data_index in range(train_x.shape[0])]
    random_data_index=np.random.choice(data_index,train_x.shape[0])#train_x.shape[0]는 15129  data_index에서 15129개 뽑기
    print(len(set(random_data_index))) #set은 중복된거 제외하고 유니크한거만 뽑음 ex) 1,2,3,2,2,3,3 ->1,2,3
    sm_train_x=train_x.iloc[random_data_index] #random하게 뽑은 인덱스의 데이터를 조회 random_data_index 번째 행조회
    sm_train_y=train_y.iloc[random_data_index]
    sm_train_x=sm.add_constant(sm_train_x,has_constant="add") #회기모델 적합(sm_train_x데이터로 적합)
    sm_model=sm.OLS(sm_train_y,sm_train_x)
    fitted_model=sm_model.fit()
    pred=fitted_model.predict(sm_test_x) #sm_test_x데이터로 예측해봄
    bagging_predict_result.append(pred)
    print(sqrt(mean_squared_error(pred,test_y))) #RMSE

9554
239994.30547408861
9546
239545.99343929227
9567
240423.22115635313
9563
239642.97091012934
9584
240129.88178164733
9606
240389.49455913328
9542
240451.21179128837
9598
239809.81813104954
9520
240235.2932057809
9621
239736.94822588068


In [27]:
bagging_predict_result[1] #리스트 1번째 처음으로 복원추출해서 만든 모형의 예측값

735      5.676984e+05
2830     7.187263e+05
4106     1.128041e+06
16218    1.489487e+06
19964    7.025307e+05
             ...     
12606    6.059318e+05
14393    6.822100e+05
6899     3.220796e+05
85       9.154704e+05
21363    4.358413e+05
Length: 6484, dtype: float64

In [28]:
bagging_predict_result[2]

735      5.621613e+05
2830     7.145261e+05
4106     1.112179e+06
16218    1.477662e+06
19964    6.954616e+05
             ...     
12606    6.041324e+05
14393    6.842960e+05
6899     3.164285e+05
85       9.077249e+05
21363    4.279677e+05
Length: 6484, dtype: float64

In [29]:
bagging_predict =[]
for lst2_index in range(test_x.shape[0]): #데이터 갯수만큼 반복 lst2_index=데이터의 인덱스 의미
    temp_predict=[]
    for lst_index in range(len(bagging_predict_result)): #len(bagging_predict_result)=10
        temp_predict.append(bagging_predict_result[lst_index].values[lst2_index]) #ex)bagging_predict_result[1]째의 1번쨔 데이터
    bagging_predict.append(np.mean(temp_predict))

In [61]:
bagging_predict

[363380.8333333334,
 570100.0,
 955625.0,
 1453000.0,
 589467.9757730016,
 277166.6666666667,
 804826.0454545455,
 416760.0,
 452202.78005538584,
 453362.73809523816,
 517980.0,
 451971.1666666667,
 291516.6666666666,
 370100.6275252525,
 479100.0,
 1437720.8333333333,
 685138.1263157895,
 930100.0,
 387700.0,
 659715.1547619046,
 360873.3333333333,
 2372000.0,
 562050.0,
 423722.2222222222,
 572966.6666666667,
 535842.8571428572,
 290501.2023809524,
 277266.6666666666,
 389043.8428571428,
 659945.0,
 590670.0,
 572230.0,
 567884.3333333334,
 669200.0,
 373420.8333333334,
 974400.0,
 966100.0,
 586500.0,
 422750.0,
 1538500.0,
 477535.0,
 295883.44,
 519154.40111111104,
 317873.3666666667,
 267781.94666666666,
 209925.0,
 241640.0,
 276981.33429216983,
 328923.0447330447,
 581380.0,
 373097.08225108223,
 294020.0,
 791616.4,
 404511.0,
 461095.59404817794,
 2216966.666666667,
 487766.6666666667,
 693125.0,
 270300.0,
 635660.4712524649,
 584263.6333333333,
 416216.6666666666,
 245386.0

In [30]:
sqrt(mean_squared_error(bagging_predict,test_y)) #bagging 앙상블모델의 rmse
#그냥회기랑 성능 비슷

239787.35336283577

### 함수사용하여(기능은 코드랑 같음) 학습 데이터를 선형 회기 모형에 적합 후 평가 데이터로 검증(Scikit-Learn)
#### linearregression 함수사용
#### bagging 함수사용

In [48]:
from sklearn. linear_model import LinearRegression
regression_model=LinearRegression()
linear_model1=regression_model.fit(train_x,train_y)


## Bagging 을 이용하여 선형 회귀 모형에 적합 후 평가 (Sampling 5번)

In [49]:
from sklearn.ensemble import BaggingRegressor
bagging_model=BaggingRegressor(base_estimator=regression_model,n_estimators=5)
linear_model2=bagging_model.fit(train_x,train_y)
predict2=linear_model2.predict(test_x)

print(sqrt(mean_squared_error(predict2,test_y))) #RMSE
#but 이걸 돌릴때마다 성능이 달라짐 랜덤 샘플링하므로

240032.97281623277


## 그렇다면 Sampling을 많이 해보자(30번)!

In [50]:
from sklearn.ensemble import BaggingRegressor
bagging_model=BaggingRegressor(base_estimator=regression_model,n_estimators=30)#30번샘플링
linear_model2=bagging_model.fit(train_x,train_y)
predict2=linear_model2.predict(test_x)

print(sqrt(mean_squared_error(predict2,test_y))) #RMSE
#but 이걸 돌릴때마다 성능이 달라짐 랜덤 샘플링하므로

239754.99012492274


## 학습 데이터를 의사결정나무모형에 적합 후 평가 데이터로 검증

In [52]:
from sklearn.tree import DecisionTreeRegressor
decision_tree_model = DecisionTreeRegressor() # 의사결정나무 모형
tree_model1 = decision_tree_model.fit(train_x, train_y) # 학습 데이터를 의사결정나무 모형에 적합
predict1 = tree_model1.predict(test_x) # 학습된 의사결정나무 모형으로 평가 데이터 예측
print("RMSE: {}".format(sqrt(mean_squared_error(predict1, test_y)))) # RMSE 결과

RMSE: 296607.29085126997


## Bagging 을 이용하여 의사결정나무모형에 적합 후 평가 (Sampling 5번,30번)

In [53]:
bagging_decision_tree_model1 = BaggingRegressor(base_estimator = decision_tree_model, # 의사결정나무 모형
                                                n_estimators = 5, # 5번 샘플링
                                                verbose = 1) # 학습 과정 표시
tree_model2 = bagging_decision_tree_model1.fit(train_x, train_y) # 학습 진행
predict2 = tree_model2.predict(test_x) # 학습된 Bagging 의사결정나무 모형으로 평가 데이터 예측
print("RMSE: {}".format(sqrt(mean_squared_error(predict2, test_y)))) # RMSE 결과

RMSE: 240399.9603692027


[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.0s finished
[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.0s finished


In [54]:
bagging_decision_tree_model2 = BaggingRegressor(base_estimator = decision_tree_model, # 의사결정나무 모형
                                                n_estimators = 30, # 30번 샘플링
                                                verbose = 1) # 학습 과정 표시
tree_model3 = bagging_decision_tree_model2.fit(train_x, train_y) # 학습 진행
predict3 = tree_model3.predict(test_x) # 학습된 Bagging 의사결정나무 모형으로 평가 데이터 예측
print("RMSE: {}".format(sqrt(mean_squared_error(predict3, test_y)))) # RMSE 결과

[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.


RMSE: 234000.2493863179


[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.7s finished
[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.0s finished


### 함수말고 코드(for문)으로 해보자

In [63]:
import random 
bagging_predict_result=[]
for _ in range(10):
    data_index=[data_index for data_index in range(train_x.shape[0])]
    random_data_index=np.random.choice(data_index,train_x.shape[0])#train_x.shape[0]는 15129  data_index에서 15129개 뽑기
    print(len(set(random_data_index))) #set은 중복된거 제외하고 유니크한거만 뽑음 ex) 1,2,3,2,2,3,3 ->1,2,3
    sm_train_x=train_x.iloc[random_data_index] #random하게 뽑은 인덱스의 데이터를 조회 random_data_index 번째 행조회
    sm_train_y=train_y.iloc[random_data_index]
    decision_tree_model=DecisionTreeRegressor()
    tree_model=decision_tree_model.fit(sm_train_x,sm_train_y)#의사결정트리
    predict_tree=tree_model.predict(test_x)
    bagging_predict_result.append(predict_tree)
    print(sqrt(mean_squared_error(predict_tree,test_y))) #RMSE
    #트리모델은 오버피팅이 심해 rmse높다

9608
279506.4973941837
9592
323050.1565084852
9541
307375.021804846
9572
293871.9910471288
9649
284857.8518277558
9556
295177.14948763564
9624
303278.9481250071
9596
296531.588106083
9577
310867.63684823376
9537
281322.3747449833


In [64]:
bagging_predict =[]
for lst2_index in range(test_x.shape[0]): #데이터 갯수만큼 반복 lst2_index=데이터의 인덱스 의미
    temp_predict=[]
    for lst_index in range(len(bagging_predict_result)): #len(bagging_predict_result)=10
        temp_predict.append(bagging_predict_result[lst_index][lst2_index]) #ex)bagging_predict_result[1]째의 1번쨔 데이터
    bagging_predict.append(np.mean(temp_predict))

In [65]:
bagging_predict

[380667.0,
 773050.0,
 924125.0,
 1647250.0,
 599297.8816102198,
 308166.6666666667,
 852986.25,
 445800.0,
 457355.0698055085,
 451128.2738095238,
 574014.2857142857,
 417854.1666666667,
 307358.3333333334,
 376801.05753968254,
 452500.0,
 1348569.3650793652,
 528476.0,
 1110250.0,
 297355.90980392153,
 674085.3571428572,
 319268.3888888889,
 2574500.0,
 591495.0,
 417307.7380952381,
 554006.6666666667,
 540741.6666666667,
 270393.4226190476,
 321993.3333333334,
 569448.75,
 676050.0,
 557530.0,
 660005.0,
 545641.2857142857,
 905120.0,
 392133.3333333334,
 1006100.0,
 1026600.0,
 609137.5,
 432956.6666666666,
 1444000.0,
 507604.5833333333,
 279791.18333333335,
 495870.63833333337,
 311550.5,
 259728.3333333333,
 213800.0,
 279430.0,
 286915.7642973856,
 322563.58730158734,
 613140.0,
 377702.65782828286,
 308940.0,
 775050.0,
 388560.0,
 450979.5451216151,
 2069200.0,
 482738.09523809527,
 693764.2857142857,
 287625.0,
 645600.0194604157,
 474984.46428571426,
 394777.0416666667,
 21

In [67]:
print(sqrt(mean_squared_error(bagging_predict,test_y)))

242941.14947896427


샘플링 갯수를 늘리면 성능이 대체로 좋아짐 하지만 어느정도까지 성능이 증가하고 그뒤로는 거의 증가하지 않으므로 너무 많이 한다고 좋은것은 아니다